## Mobility Robustness Optimization (MRO)

Takes in new observation data to train or update the bayesian digital twin model. It processes the input data and updates the model to better reflect the current network conditions.

Then MRO optimizes the mobility robustness by solving the underlying problem using the trained model: finding optimal `HYST` and `TTT`. There are two solve approach shown: 

- Simple MRO
- Reinforced MRO.

In [ ]:
import sys
from pathlib import Path
sys.path.append(f"{Path().absolute().parent}")

In [ ]:
import pandas as pd
import numpy as np
from apps.mobility_robustness_optimization.simple_mro import SimpleMRO
from apps.mobility_robustness_optimization.mro_rl import ReinforcedMRO

# Assuming user has only UE data, without received power

use the following example `topology` and `mobility_model_params` to initiate MRO. Showcasing **Simple MRO** solving approach.


*unzip the `data/mro_data.zip file` to get `data/mro_data/` folder*

In [ ]:
topology = pd.read_csv('data/mro_data/mro_topology.csv')
topology

In [ ]:
mobility_model_params = {
    "ue_tracks_generation": {
            "params": {
                "simulation_duration": 3600,
                "simulation_time_interval_seconds": 0.01,
                "num_ticks": 50,
                "num_batches": 1,
                "ue_class_distribution": {
                    "stationary": {
                        "count": 10,
                        "velocity": 0,
                        "velocity_variance": 1
                    },
                    "pedestrian": {
                        "count": 5,
                        "velocity": 2,
                        "velocity_variance": 1
                    },
                    "cyclist": {
                        "count": 5,
                        "velocity": 5,
                        "velocity_variance": 1
                    },
                    "car": {
                        "count": 12,
                        "velocity": 20,
                        "velocity_variance": 1
                    }
                },
                "lat_lon_boundaries": {
                    "min_lat": -90,
                    "max_lat": 90,
                    "min_lon": -180,
                    "max_lon": 180
                },
                "gauss_markov_params": {
                    "alpha": 0.5,
                    "variance": 0.8,
                    "rng_seed": 42,
                    "lon_x_dims": 100,
                    "lon_y_dims": 100,
                    "// TODO": "Account for supporting the user choosing the anchor_loc and cov_around_anchor.",
                    "// Current implementation": "the UE Tracks generator will not be using these values.",
                    "// anchor_loc": {},
                    "// cov_around_anchor": {}
            }
        }
    }
}

<b>Optionally,</b> use mobility model to get `alpha` of your data and set it to params.

In [ ]:
# [OPTIONAL] run this cell to get alpha calculated from the data into mobility_model_params

from radp.digital_twin.mobility.param_regression import get_predicted_alpha

# 20 UEs x 100 ticks = 2000 rows
ue_data = pd.read_csv("data/mro_data/UE_data_20UE_100ticks.csv") # change this to the data you want to use
ue_data = ue_data.rename(columns={'latitude': 'lat', 'longitude': 'lon'})

# set random initial alpha
alpha0 = np.random.choice(np.arange(0, 1.1, 0.1))

alpha = get_predicted_alpha(ue_data, alpha0 = alpha0, seed = 42)

print(f"Learned alpha: {alpha}\n")

mobility_model_params["ue_tracks_generation"]["params"]["gauss_markov_params"]["alpha"] = alpha

In [ ]:
mro = SimpleMRO(mobility_model_params, topology)

In [ ]:
# initially bayesian_digital_twins is empty
print(f"bayesian_digital_twins: {mro.bayesian_digital_twins}")

- prepare `new_data` for training/updating `bayesian_digital_twins`

    - `new_data` should have received power data in cartesian df format
    - if only have [latitude, longitude], call `preprocess_ue_data()` from RADP library

In [ ]:
new_data_no_rxpwr = pd.read_csv("data/mro_data/UE_data_20UE_100ticks.csv") # change this to the data you want to use

# 20 UEs x 100 ticks = 2000 rows
new_data_no_rxpwr = new_data_no_rxpwr.loc[:, ['longitude', 'latitude']]
new_data_no_rxpwr.head()

If have data without received power information, then call `preprocess_ue_data()` from RADP library.

In [ ]:
from notebooks.radp_library import preprocess_ue_data

input_data = preprocess_ue_data(new_data_no_rxpwr, topology)
input_data.head()

In [ ]:
# train bayesian_digital_twins from scratch
mro.train_or_update_rf_twin(new_data=input_data)

In [ ]:
mro.bayesian_digital_twins # bayesian_digital_twins is trained for each cell_id

call `solve()` method to get optimized `HYST` and `TTT`

In [ ]:
# adjust n_epochs for better performance
mro.solve(n_epochs=100)

can save trained/updated `bayesian_digital_twins`

In [ ]:
saving_dir_relative_path = "data/mro_data/"

mro.save_bdt(saving_dir_relative_path) # True indicates save is successful

can load this `bayesian_digital_twins` when needed

In [ ]:
mro.bayesian_digital_twins = {} # bayesian_digital_twins is empty again

pkl_file_path = "data/mro_data/digital_twins.pkl"
mro.load_bdt(pkl_file_path) # True indicates load is successful

let's try updating the `bayesian_digital_twins` with new observations

In [ ]:
# loading new UE data assuming user have new observations now.

# new observations with [latitude, longitude] --> preprocess_ue_data(new_data, topology) --> new_obeservations
new_obeservations = pd.read_csv("data/mro_data/UE_data_new_observations_20UE_100ticks.csv")
input_data = new_obeservations.copy()
input_data.head()

In [ ]:
# update bdt with new observations, calling train_or_update_rf_twin() again
mro.train_or_update_rf_twin(input_data)

can solve with updated `bayesian_digital_twins`

In [ ]:
# adjust n_epochs for better performance
mro.solve(n_epochs=100)

# Assuming user has UE data, including received power information

use the following example `topology` and `mobility_model_params` to initiate MRO. Showcasing **Reinforced MRO** solving approach.

In [ ]:
rl_mro = ReinforcedMRO(mobility_model_params, topology)

In [ ]:
print(f"bayesian_digital_twins: {rl_mro.bayesian_digital_twins}", end='\n\n') # bayesian_digital_twins is empty initially

pkl_file_path = "data/mro_data/digital_twins.pkl" # run previous section to have this file
rl_mro.load_bdt(pkl_file_path) # True indicates load is successful

In [ ]:
rl_mro.bayesian_digital_twins

- load `new_data` for training/updating `bayesian_digital_twins`

    - `new_data` should have received power data in cartesian df format

In [ ]:
# 20 UEs x 50 ticks x 3 cells cartesian = 3000 rows
ue_data_with_rxpower = pd.read_csv("data/mro_data/UE_data_with_rxpower_20UE_50ticks.csv")
input_data = ue_data_with_rxpower.copy()

input_data

In [ ]:
# Dummy solve call to avoid fantasy observation error: ensuring all test independent caches exist
rl_mro.solve(total_timesteps=2)

In [ ]:
# update bdt with new data
rl_mro.train_or_update_rf_twin(input_data)

In [ ]:
# adjust total_timesteps for better performance
rl_mro.solve(total_timesteps=100)